In [1]:
import time
import scipy
import pickle
import scipy.io
import numpy as np
from scipy.linalg import norm
import matplotlib.pyplot as plt

In [32]:
data = scipy.io.loadmat('./data/syn_1000_1200_10_50.mat')
W = data['W']
with open('./data/1000_1200_sol.pckl', 'rb') as f:
    sol = pickle.load(f)

In [25]:
def obj_func(x):
    return -np.sum(np.log(W.dot(x)))

In [26]:
n, p = W.shape
x = np.ones(p) / p
n_steps = 3000
printst = 100
terminate_tol = 1e-8
sub_tol = 0.25 * terminate_tol
steps_tol = 0.05
Lest = 1

In [27]:
s_deactive = 0

if Lest == 1:
    #Estimate Lipschitz Constant
    Denom = W.dot(np.ones(p)) / p
    RatH = 1 / (Denom**2)    
    dirr = np.ones(p)
    for Liter in range(1, 16):
        Dir = W.T.dot(RatH * (W.dot(dirr)))
        dirr = Dir / norm(Dir)
    Hd = W.T.dot(RatH * (W.dot(dirr)))
    dHd = dirr.dot(Hd)
    L = dHd / (dirr.dot(dirr))

In [28]:
def fn_portf_proxsplx(y):
    m = len(y)
    bget = False
    s  = sorted(y, reverse=True)
    tmpsum = 0
    for i in range(m - 1):
        tmpsum = tmpsum + s[i]
        tmax = (tmpsum - 1) / (i + 1)
        if tmax >= s[i+1]:
            bget = True
            break
    if not bget:
        tmax = (tmpsum + s[m - 1] - 1) / m
    x = np.maximum(y - tmax, 0)
    return x

In [29]:
def fn_portf_fw_bi(Fval, a, b, tol, n):
    Vl = Fval(a)
    Vr = Fval(b)
    Sol = 2 / (n + 1)
    for i in range(1, 5001):
        if abs(Vl) <= tol:
            Sol = a
            #print( 'Binary search stops after %2d iterations\n' % i)
            break

        if abs(Vr) <= tol:
            Sol = b
            #print( 'Binary search stops after %2d iterations\n' % i)
            break

        c = (a + b) / 2
        Vc = Fval(c)
        if Vc * Vl > 0:
            a = c
            Vl = Vc
        else:
            b = c
            Vr = Vc
    return Sol, i

In [30]:
err, times, points = [], [], []
points.append(x)

print('********* Algorithm starts *********\n');

int_start = time.time()
for i in range(1, n_steps + 1):   
    
    start = time.time()
    
    # Compute the denominator.
    denom = W.dot(x)
    
    # Evaluate the gradient.
    ratG = 1 / denom # n by 1 vector
    Grad = -W.T.dot(ratG) # p by 1 vector
    
    if i == 1:
        nume = W.dot(Grad)
        Gradtau = lambda t: np.sum(nume / (denom - t*nume))
        tau, _ = fn_portf_fw_bi(Gradtau, 0, 1, 1e-7, i)
    else:
        Gradif = Grad - Grad_pre;
        tau = (diffx.dot(Gradif)) / (Gradif.dot(Gradif)) 
        
    x_nxt_tmp = x - tau * Grad
    x_nxt = fn_portf_proxsplx(x_nxt_tmp)

    diffx = x_nxt - x
    
    # solution value stop-criterion    
    nrm_dx = norm(diffx)
    rdiff = nrm_dx / max(1.0, norm(x))
    err.append(rdiff)
    
    # Check the stopping criterion
    if (rdiff <= terminate_tol) and i > 1:
        print('Convergence achieved!')
        print('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n' % (i, tau, rdiff))
        break
    
    x = x_nxt
    Grad_pre = Grad
    
    end = time.time()
    times.append(end - start)
    points.append(x)
    if i % printst == 0 or i == 1:
        print('iter = %4d, stepsize = %3.3e, rdiff = %3.3e\n' % (i, tau, rdiff))

int_end = time.time()

********* Algorithm starts *********

iter =    1, stepsize = 1.000e+00, rdiff = 9.996e-01

iter =  100, stepsize = 2.665e-04, rdiff = 2.029e-04

iter =  200, stepsize = 2.859e-04, rdiff = 2.036e-04

iter =  300, stepsize = 3.076e-04, rdiff = 2.039e-04

iter =  400, stepsize = 3.317e-04, rdiff = 2.035e-04

iter =  500, stepsize = 3.586e-04, rdiff = 2.024e-04

iter =  600, stepsize = 3.885e-04, rdiff = 2.005e-04

iter =  700, stepsize = 4.217e-04, rdiff = 1.974e-04

iter =  800, stepsize = 4.584e-04, rdiff = 1.931e-04

iter =  900, stepsize = 4.988e-04, rdiff = 1.873e-04

iter = 1000, stepsize = 5.427e-04, rdiff = 1.799e-04

iter = 1100, stepsize = 5.900e-04, rdiff = 1.708e-04

iter = 1200, stepsize = 6.400e-04, rdiff = 1.600e-04

iter = 1300, stepsize = 6.918e-04, rdiff = 1.476e-04

iter = 1400, stepsize = 7.442e-04, rdiff = 1.339e-04

iter = 1500, stepsize = 7.957e-04, rdiff = 1.193e-04

iter = 1600, stepsize = 8.447e-04, rdiff = 1.043e-04

iter = 1700, stepsize = 8.898e-04, rdiff = 8

In [33]:
norm(x - sol) / max(1, norm(sol))

0.0019005556139938924

In [34]:
int_end - int_start

11.673953294754028

In [15]:
with open('./AGD_times.pckl', 'wb') as f:
    pickle.dump(times, f)